In [1]:
from two_stage_rag_pipeline import *
#from rag_pipeline import *
#from test_pipeline import *
import os
import pandas as pd
from uuid import uuid4
from tqdm.notebook import tqdm
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall,
)
# 최신 버전 기준 (ragas >= 0.1.2)
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas import EvaluationDataset
from datasets import Dataset as HFDataset
# 초기 세팅
vectorstore = initialize_pinecone()
conversation_chain = initialize_conversation(vectorstore)

C:\Users\samsung-user\anaconda3\envs\boo_chat\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
session_id = "test-session"  # 또는 str(uuid4()) 사용

doc_list = load_all_documents()
texts = [doc.page_content for doc in doc_list]
bm25_retriever = BM25Retriever.from_texts(texts)
bm25_retriever.k = 30

pinecone_retriever = vectorstore.as_retriever(search_kwargs={"k": 30})
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, pinecone_retriever], weights=[0.3, 0.7])

reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
reranker = CrossEncoderReranker(model=reranker_model, top_n=15)
compression_retriever = ContextualCompressionRetriever(base_compressor=reranker, base_retriever=ensemble_retriever)

In [3]:
import os
from dotenv import load_dotenv
from datasets import Dataset


def main():
    load_dotenv()

    # 저장 폴더 준비
    os.makedirs("data", exist_ok=True)

    # 질문 리스트
    questions = [
        "월요일 7, 8교시에 들을 수 있는 교양 있으면 알려줘.",
        "글스산 졸업하면 뭐해?",
        "졸업하기 전에 무슨 영어성적 제출해야 된다고 했던거 같은데 그건 뭐야?",
        "후기 이중에 대해 설명해줘.",
        "김정환 교수님 연구실 어디야?",
        "컴퓨팅사고를 AI융합전공 이중과목으로 변경해야되는데 이수구분변경 신청 어떻게 해?",
        "감기로 병원다녀왔는데 진료확인서로 유고결석 가능해?",
        ("나 정보통신공학과인데 혹시 졸업시험 대신 졸업논문을 선택해도 될까? "
         "논문을 쓰게 되면 머신러닝 관련해서 작성하고 싶은데 이와 관련해서 상담할 수 있는 교수님이 계시면 이메일 좀 알려줘."),
        "일통 FLEX 시험 점수 제출 기간 알고 싶어.",
        "글스산 세부전공 신청해야되는데, 신청기간이랑 방법 알 수 있어? 그리고 결과는 언제나와?"
    ]

    # Ground‐truth 리스트
    reference = [
        ["월요일 7,8교시에 들을 수 있는 교양 과목에는 '인간과 사회', '대중문화 예술의 이해', '비즈니스협상' 등이 있습니다."],
        ["글로벌스포츠산업학부를 졸업하면 주로 다음과 같은 분야에서 활동하게 됩니다:\n"
         "1. 스포츠매니지먼트 및 이벤트 기획\n"
         "2. 스포츠 마케팅 및 홍보\n"
         "3. 스포츠 데이터 분석가\n"
         "4. 스포츠 미디어\n"
        "이 외에도 졸업 후에는 대학원 진학이나 관련 분야의 연구를 진행할 수 있는 기회도 있습니다."
        ],
        ["졸업하기 전에 제출해야 하는 영어 성적은 외국어 인증을 위한 것으로, 다음과 같은 기준을 충족해야 합니다:\n\n1." 
        "**인증 시험 성적**: 제출 시점 기준 유효기간(2년) 이내의 인증 시험 성적이 필요합니다. 인정되는 시험은 다음과 같습니다:\n"   
        "- TOEIC\n   - TOEFL\n   - IELTS\n   - FLEX 외국어 능력 시험 등\n\n2." 
        "**대체 과정 수료증**: 인증 시험 성적 대신 외국어 연수 평가원 또는 외국어 교육센터에서 제공하는 대체 과정을 이수한 경우," 
        "해당 수료증을 제출할 수 있습니다.\n\n3. **제출 마감일**: 졸업하고자 하는 최종 학기 말까지 반드시 제출해야 하며," 
        "재학 기간 동안 1회 외국어 인증 신청서를 제출할 수 있습니다.\n\n"
        "정확한 기준과 제출 방법은 학사종합지원센터에 문의하시거나, 해당 과목의 담당 교수님께 확인하시기 바랍니다."],
         ["후기 이중전공은 부전공 이수자가 정규 8학기 내에 제1전공 졸업학점(부전공 21학점 포함)을 모두 취득한 후, "
         "1~2학기 추가 수학하여 해당 부전공의 이중전공화에 필요한 나머지 학점을 취득함으로써, "
         "제1전공과 이중전공 학위를 동시에 취득하는 제도입니다.\n"
        "신청은 매년 4월에 한 번만 가능하며, 신청 자격은 현재 7~8학기 이수 중인 학생으로, 부전공을 이수 중인 경우에 해당합니다.\n" 
        "추가학기를 등록해야 하며, 등록금은 후기 이중전공 학과의 등록금으로 납부해야 합니다. 더 궁금하신 점이 있으시면 말씀해 주시기 바랍니다."],
        ["김정환 교수님의 연구실은 어문관 435호에 위치해 있습니다. 추가로 궁금하신 점이 있으시면 언제든지 말씀해 주세요."],
        ["컴퓨팅사고를 AI융합전공 이중과목으로 변경하시려면, 이수구분 변경 신청서를 작성하신 후," 
        "해당 전공 소속 단과대학의 담당 선생님에게 이메일로 제출하시거나 직접 방문하여 제출하셔야 합니다.\n"
        "신청은 졸업학기 졸업사정일 전까지 가능하며, 제출 후 3~4일 이내에 종합정보시스템에서 변경 결과를 확인하실 수 있습니다.\n" 
        "만약 변경이 되지 않았거나 문제가 발생한 경우, 즉시 담당 선생님에게 연락하여 확인하셔야 합니다."
        "자세한 사항은 학사종합지원센터의 공지사항을 참고하시기 바랍니다. 도움이 필요하시면 언제든지 문의해 주세요."],
        ["감기로 병원에 다녀오신 경우, 진료확인서로 유고결석을 인정받기 위해서는 몇 가지 조건이 있습니다."
         "일반적으로 의원에서 발급받은 서류는 유고결석 인정에 해당되지 않으며, 병원급 이상의 의료기관에서 발급된 증빙서류가 필요합니다."
         "따라서, 감기로 병원급 의료기관에서 진료를 받으셨다면 해당 증빙서류를 제출하시면 유고결석이 가능할 수 있습니다.\n"
        "정확한 사항은 담당 교수님께 문의하시고, 필요한 서류를 제출하시는 것이 좋습니다. 추가적인 질문이 있으시면 언제든지 말씀해 주시기 바랍니다."],
        ["졸업시험 대신 졸업논문을 선택하실 수 있습니다."
         "졸업논문을 작성하실 경우, 머신러닝 관련 주제로 진행하고 싶으시다면, 관련 상담을 위해 한희일 교수님께 연락하실 수 있습니다\n"
         "한희일 교수님의 이메일은 hihahn@hufs.ac.kr 입니다."],
        ["졸업시험 점수 제출 기간은 공지사항에 따라 각 학과 사무실로 문의해야 합니다. 추가적인 질문이 있으시면 말씀해주세요."],
        ["글로벌스포츠산업학부의 세부전공 신청에 대한 정보는 다음과 같습니다.\n"
         "신청기간: 2025년 5월 26일(월) 10:00부터 5월 30일(금) 16:00까지입니다. 신청기간 이후에는 추가 신청이나 변경이 절대 불가하니 유의하시기 바랍니다.\n"
         "신청방법: 종합정보시스템에 접속하셔서 전공/교직관리 메뉴에서 1전공 신청을 하시면 됩니다.\n"
         "배정결과: 배정 확정은 2025년 7월 중에 이루어질 예정입니다. 성적 반영이 완료된 후에 결과를 확인하실 수 있습니다.\n"
         "추가적인 문의사항이 있으시면 학사종합지원센터에 연락하시기 바랍니다."]
    ]

    # RAG 추론 및 컨텍스트 수집
    answers = []
    contexts = []
    session_id = "test-session"  # 또는 str(uuid4()) 사용
    
    for q in questions:
        result = conversation_chain.invoke(
            {"input": q},
            config={"configurable": {"session_id": session_id}}
        )
    
        answers.append(result["answer"])  # ⬅️ 문자열만 저장

        
        # 호출 간에 딜레이 주기 (예: 2초)
        time.sleep(5)
        
        docs = compression_retriever.invoke(q)
        contexts.append([doc.page_content for doc in docs])

        time.sleep(1)
        
    # Dataset 생성
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "reference": reference
    }
    dataset = Dataset.from_dict(data)


    dataset.save_to_disk("ragas_2stage_data")

    print("Dataset saved to data")
    print(dataset)

if __name__ == "__main__":
    main()

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset saved to data
Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 10
})


In [5]:
from datasets import load_from_disk
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    faithfulness,
    answer_relevancy,
)

def main():
    # 저장된 dataset 불러오기
    dataset = load_from_disk("ragas_2stage_data")

    def convert_reference(example):
        ref = example["reference"]
        if isinstance(ref, list):
            example["reference"] = ref[0]
        return example
    
    # 3. map으로 전체에 적용
    dataset = dataset.map(convert_reference)

    # 평가 실행
    result = evaluate(
        dataset=dataset,
        metrics=[
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy,
        ],
    )

    # 결과 확인 및 저장
    print(result)
    df = result.to_pandas()
    df.to_csv("data/evaluation_results_2stage.csv", index=False)
    print("Evaluation results saved to evaluation_results.csv")

if __name__ == "__main__":
    main()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

{'context_precision': 0.6141, 'context_recall': 0.6817, 'faithfulness': 0.7355, 'answer_relevancy': 0.1556}
Evaluation results saved to evaluation_results.csv


In [6]:
import pandas as pd
df = pd.read_csv('data/evaluation_results_2stage.csv')
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,"월요일 7, 8교시에 들을 수 있는 교양 있으면 알려줘.",['통해 다채로운 한국문화 관련 세부 전공을 개척해 나갈 수 있습니다. \nQ3. ...,"월요일 7, 8교시에 들을 수 있는 교양 과목은 다음과 같습니다:\n\n1. **종...","월요일 7,8교시에 들을 수 있는 교양 과목에는 '인간과 사회', '대중문화 예술의...",0.000000,0.000000,0.894737,0.792193
1,글스산 졸업하면 뭐해?,['3학년 통번역을 중점적으로 다루며 졸업시험을 위한 FLEX 준비도 시\n작하게...,'글스산'은 '글로벌스포츠산업학부'를 의미합니다. 졸업 후 글로벌스포츠산업학부의 졸...,글로벌스포츠산업학부를 졸업하면 주로 다음과 같은 분야에서 활동하게 됩니다:\n1. ...,0.283333,1.000000,0.916667,0.763527
2,졸업하기 전에 무슨 영어성적 제출해야 된다고 했던거 같은데 그건 뭐야?,['## [heading1] ##\n# 2. 졸업시험 / 논문\n\n## [list...,졸업하기 전에 제출해야 하는 영어 성적은 외국어 인증과 관련된 사항입니다. 영어 통...,"졸업하기 전에 제출해야 하는 영어 성적은 외국어 인증을 위한 것으로, 다음과 같은 ...",0.712629,0.666667,0.714286,0.000000
3,후기 이중에 대해 설명해줘.,"['정하는 시간을 갖게 됩니다.\n2학년 한국문학, 한국역사, 한국문화로 나눠진 ...",'후기 이중전공'은 부전공 이수자가 정규 8학기 내에 제1전공 졸업학점(부전공 21...,후기 이중전공은 부전공 이수자가 정규 8학기 내에 제1전공 졸업학점(부전공 21학점...,0.491429,0.750000,0.800000,0.000000
4,김정환 교수님 연구실 어디야?,['김정환 교수님은 루마니아학과 학과 소속입니다. 교수 직위를 맡고 있습니다. 이메...,"김정환 교수님은 루마니아학과 소속이며, 연구실은 어문관 435호에 위치해 있습니다....",김정환 교수님의 연구실은 어문관 435호에 위치해 있습니다. 추가로 궁금하신 점이 ...,1.000000,0.500000,0.666667,0.000000
5,컴퓨팅사고를 AI융합전공 이중과목으로 변경해야되는데 이수구분변경 신청 어떻게 해?,"[""[서울] 이수구분 변경 신청 방법 안내\n\n[서울] 이수구분 변경 신청 방법 ...",'컴퓨팅사고' 과목을 AI융합전공 이중과목으로 변경하기 위해서는 이수구분 변경 신청...,"컴퓨팅사고를 AI융합전공 이중과목으로 변경하시려면, 이수구분 변경 신청서를 작성하신...",1.000000,0.600000,0.666667,0.000000
6,감기로 병원다녀왔는데 진료확인서로 유고결석 가능해?,['## [paragraph] ##\n나. 처리절차\n\n## [paragraph]...,"감기로 병원에 다녀오신 경우, 일반 의원에서 발급한 진료확인서는 유고결석으로 인정되...","감기로 병원에 다녀오신 경우, 진료확인서로 유고결석을 인정받기 위해서는 몇 가지 조...",1.000000,0.800000,0.750000,0.000000
7,나 정보통신공학과인데 혹시 졸업시험 대신 졸업논문을 선택해도 될까? 논문을 쓰게 되...,['## [heading1] ##\n# 2. 졸업시험 / 논문\n\n## [list...,정보통신공학과에서는 졸업시험 대신 졸업논문을 선택할 수 있습니다. 졸업논문을 작성하...,"졸업시험 대신 졸업논문을 선택하실 수 있습니다.졸업논문을 작성하실 경우, 머신러닝 ...",0.453399,1.000000,0.571429,0.000000
8,일통 FLEX 시험 점수 제출 기간 알고 싶어.,"['A0, 801점 이상 A+), 1회에 한함 - 1전공과 동일언어 학점 불인정 -...",일본어통번역학과(일통)의 FLEX 시험 점수 제출 기간은 해당학과 사무실에 문의하셔...,졸업시험 점수 제출 기간은 공지사항에 따라 각 학과 사무실로 문의해야 합니다. 추가...,0.654500,0.500000,0.750000,0.000000
9,"글스산 세부전공 신청해야되는데, 신청기간이랑 방법 알 수 있어? 그리고 결과는 언제나와?",['불가\n\n① 모든 과목은 수강신청(변경)기간에 학생 본인이 온라인으로 수강신청...,글로벌스포츠산업학부(글스산)의 세부전공 신청 기간과 방법은 다음과 같습니다:\n\n...,글로벌스포츠산업학부의 세부전공 신청에 대한 정보는 다음과 같습니다.\n신청기간: 2...,0.545406,1.000000,0.625000,0.000000


In [12]:
# 보기 좋게 출력하는 함수
def print_ragas_eval(df):
    for i, row in df.iterrows():
        print(f"--- Row {i} ---")
        print(f"[User Input]\n{row['user_input']}\n")
        print(f"[Response]\n{row['response']}\n")
        #print(f"[Reference]\n{row['reference']}\n")
        print(f"[Faithfulness]: {row['faithfulness']:.3f}")
        print(f"[Answer Relevancy]: {row['answer_relevancy']:.3f}")
        print("=" * 40)

# 사용 예시 (상위 5개만 출력)
print_ragas_eval(df)

--- Row 0 ---
[User Input]
월요일 7, 8교시에 들을 수 있는 교양 있으면 알려줘.

[Response]
월요일 7, 8교시에 들을 수 있는 교양 과목은 다음과 같습니다:

1. **종교철학입문**
   - 교수: 조명동
   - 강의실: 2503호
   - 학점: 2
   - 전공필수 과목 아님
   - 온라인강의 아님
   - P/F 평가 과목 아님
   - 원어강의 아님

2. **인문적사유와글쓰기**
   - 교수: 한상효
   - 강의실: 2303-2호
   - 학점: 2
   - 전공필수 과목 아님
   - 온라인강의 아님
   - P/F 평가 과목 아님
   - 원어강의 아님

이 과목들을 고려해 보시기 바랍니다. 혹시 더 도와드릴까요?

[Faithfulness]: 0.895
[Answer Relevancy]: 0.792
--- Row 1 ---
[User Input]
글스산 졸업하면 뭐해?

[Response]
'글스산'은 '글로벌스포츠산업학부'를 의미합니다. 졸업 후 글로벌스포츠산업학부의 졸업생들은 다양한 진로를 선택할 수 있습니다. 

졸업 후 진로는 다음과 같습니다:

1. **스포츠 매니지먼트 및 이벤트 기획**: 스포츠 클럽, 리그, 대회, 이벤트 등을 기획하고 운영하는 직무에 진출할 수 있습니다.
2. **스포츠 마케팅 및 홍보**: 스포츠 산업에서 제품, 서비스, 이벤트 등을 마케팅하고 홍보하는 업무를 수행할 수 있습니다.
3. **스포츠 데이터 분석가**: 스포츠 경기에서 발생하는 다양한 데이터를 수집하고 분석하여 팀이나 선수의 성과를 향상시키는 데 필요한 정보를 제공하는 역할을 할 수 있습니다.

이 외에도 관련 분야에서 다양한 직무를 수행할 수 있으며, 대학원 진학이나 창업의 기회도 있습니다. 졸업 후 진로에 대한 더 구체적인 정보는 담당 교수님께 문의하시기 바랍니다. 혹시 더 도와드릴까요?

[Faithfulness]: 0.917
[Answer Relevancy]: 0.764
--- Row 2 ---
[

In [14]:
print(df['retrieved_contexts'][2])

['## [heading1] ##\n# 2. 졸업시험 / 논문\n\n## [list] ##\n- 가. 1전공, 이중전공, 제2전공(2006학번 이전)은 반드시 졸업시험이나 논문을 통과하여야\n- 졸업이 가능함\n- 나. 매 학기 중반(4~5월, 10~11월)에 졸업시험 및 논문작성/심사 등을 진행하며, 학과별,\n- 전공별 조건 및 일정은 각 학과사무실에 확인하여 기간 내 조치토록 함\n- 다. 졸업시험/논문에 합격 후 졸업 시까지 그 자격이 유효하나, 합격 후 제적처리가 되는\n- 경우에는 합격이 취소됨\n- 라. 졸업시험FLEX와 외국어인증은 별도이므로 각각 요건을 확인하고 응시 / 처리하여야 함\n\n## [paragraph] ##\n3. 외국어인증(졸업인증) : 입학과 동시에 제출 가능', '을 공부해 두는 것이 좋습니다. 졸업 후 취업 또는 진학에 관계없이 영어 \n구사 능력은 항시 요구되므로 대학 입학 이후부터 읽기, 듣기, 말하기, \n쓰기 모든 분야에서 영어 능력을 지속적으로 계발하는 것이 매우 중요\n합니다. 고학년 기간에는 이론적인 지식이 아니라 연구실 인턴을 통해 \n관심분야의 연구를 경험해보며 자신의 진로를 준비하는 것이 좋습니다.\n졸업 후 진로 \n-  기업체: 석유화학, 반도체/전자, 에너지, 신소재, 재료, 바이오, 의료 및 \n제약, 화장품 및 식품 분야\n-  정부 및 공공기관: 국가/지방 공무원 및 정부 산하 공기업\n-  학자 및 연구직: 대학 교수, 국공립연구소 및 정부 출연 연구소, 기\n업 연구소\n-  기타: 교사(교직이수), 창업\nQ&A', '2. 졸업시험 / 논문 ······································································ 17\n3. 외국어인증(졸업인증) ······································································ 17\n4. 기타 졸업관련 사항 ···················

In [7]:
# 1. 평균값 구하고 전치
df1 = df.mean(numeric_only=True, skipna=True).reset_index()
df1 = df1.T

# 2. 첫 번째 행을 컬럼명으로 설정
df1.columns = df1.iloc[0]

# 3. 첫 번째 행(이전 헤더로 썼던 행) 제거
df1 = df1.drop(df1.index[0]).reset_index(drop=True)

# 4. 모델 컬럼 추가 및 인덱스 삭제
df1 = df1.reset_index(drop=True)
df1['model'] = '2stage'


In [8]:
df1 

index,context_precision,context_recall,faithfulness,answer_relevancy,model
0,0.61407,0.681667,0.735545,0.155572,2stage


In [9]:
df1.to_csv('data/2stage.csv',encoding='utf-8',index=False)